## ERT回帰モデルを使った交通量予測

ロサンゼルススタジアムで開催された野球の試合の間に通行した車の量を数えたデータを扱う

トラフィックデータはこんな感じ

曜日, 時間, 対戦相手, 試合中かどうか(yes/no), 通った車の数
```
Tuesday,00:00,San Francisco,no,3
Tuesday,00:05,San Francisco,no,8
Tuesday,00:10,San Francisco,no,10
：
：
```

情報から車の通行量を予測できるようにする。

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, mean_absolute_error
from sklearn import model_selection, preprocessing
from sklearn.ensemble import ExtraTreesRegressor

# 変数定義
input_file = './data/traffic_data.txt'
data = []

# ファイルを読み込む
with open(input_file, 'r') as f:
    # 1行づつ読み込む。lineには "Tuesday,00:00,San Francisco,no,3\n\n" な感じで入る。
    for line in f.readlines():
        # line[:-1]で改行コードを取り除いて、","で分割したリストをitemsに入れる
        items = line[:-1].split(',')
        # dataにitemsを追加していく
        data.append(items)

# Numpy配列に変換する
data = np.array(data)

# ラベル作り
label_encoder = []
X_encoded = np.empty(data.shape)

# dataからそれぞれインデックス番号と値を取り出す
for i, item in enumerate(data[0]):
    # 数値ならそのまま入れる
    if item.isdigit():
        X_encoded[:, i] = data[:, i]
    # 数値以外ならテキストを数値にラベル化する
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(data[:, i])

# X_encodedは[ 5.  4. 13.  0.  1.]な感じ
X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)


# 訓練用データを75%、テスト用データは25%
# random_stateは結果に再現性を求める時に指定すると良い。
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=5)

# ERT回帰のパラメーター設定と回帰学習器を作成
params = {'n_estimators': 100, 'random_state': 0, 'max_depth': 4}
regressor = ExtraTreesRegressor(**params)

# 訓練
regressor.fit(X_train, y_train)

# 検証
y_pred = regressor.predict(X_test)

print("Mean absolute error:", round(mean_absolute_error(y_test, y_pred), 2)) 

[[ 5.  4. 13.  0.  1.]
 [ 5.  5. 13.  0.  4.]
 [ 5.  6. 13.  0.  9.]
 [ 5.  7. 13.  0.  4.]
 [ 5.  8. 13.  0.  6.]
 [ 5.  9. 13.  0. 13.]]
Mean absolute error: 7.42


In [17]:
# 適当な入力値を作る
test_datapoint = ['Saturday', '10:20', 'Atlanta', 'no'] 
test_datapoint_encoded = [0] * len(test_datapoint) 
count = 0 

# 入力パラメータを先ほど作成したラベルエンコーダーを元に作成する
for i, item in enumerate(test_datapoint): 
    if item.isdigit(): 
        test_datapoint_encoded[i] = int(test_datapoint[i]) 
    else: 
        test_datapoint_encoded[i] = int(label_encoder[count].transform(
                                        [test_datapoint[i]])[0]) 
        count = count + 1 

# Numpy配列に変換する
test_datapoint_encoded = np.array(test_datapoint_encoded) 

# 予測値を出す
print("Predicted traffic:", int(regressor.predict([test_datapoint_encoded])[0])) 

Predicted traffic: 26
